In [3]:
import rdflib as rdf
import random
import textdistance
import re
from xml.dom import minidom
import sys as sys


In [4]:
def get_functional_properties( graph ):
    
    candidates_properties= set(graph.predicates()) # toutes les propriétés sont candidates
    
    for subject in graph.subjects():
        predicate_occurences=dict()
        for i in graph.predicate_objects(subject):
            if i[0] not in predicate_occurences:
                predicate_occurences[i[0]]=1
            else : 
                predicate_occurences[i[0]]+=1
        for p, n in predicate_occurences.items(): 
            if n > 1:
                if p in candidates_properties:
                    candidates_properties.remove(p)
                    
    return candidates_properties

In [5]:
#Property functionality degree : fun(p)=nombre de fois  un suject possède cette propriété/ nombre de fois que le propriété apparait
#autre idée: récupérer uniquement les propriétés qui n'apparaissent que une fois par sujet

In [6]:
def get_functional_properties( graph, threshold ):

    nomminateur=dict() #dictionnaire avec pour chaque propriété, le nombre de sujet qui possèdent cette propriété
    
    for p in set(graph.predicates()):
        
        for s in set(graph.subjects()):
            if len(list(graph.objects(subject=rdf.URIRef(s), predicate=rdf.URIRef(p)))) >=1:
                if p not in nomminateur:
                    nomminateur[p]=1
                else:
                    nomminateur[p]+=1         
    
    denominateur=dict() # dictionnaire avec pour chaque propriété, le nombre de fois quelle apparait 
    
    for p in list(graph.predicates()):
        if p not in denominateur:
            
            denominateur[p]=1
        else:
            denominateur[p]+=1
    
    functional_properties=set()
    for key,value in nomminateur.items():
        if value/denominateur[key]>= threshold :
            functional_properties.add(key)
            
    return functional_properties

In [7]:
def is_date (date):
    if re.match("^[0-9][0-9][0-9][0-9]\-[0-9][0-9]\-[0-9][0-9]$", date) :
        return True
    if re.match("^[0-9][0-9][0-9][0-9]\-[0-9][0-9]$", date) :
        return True
    if re.match("^[0-9][0-9][0-9][0-9]$", date) :
        return True
    if re.match("^[0-9]+\/[0-9]+\/[0-9]+$", date) :
        return True
    if re.match("[a-zA-Z]*\s[0-9]+\,\s[0-9][0-9][0-9][0-9]$", date):
        return True
    return False

In [8]:
mois={'Jan':'01','Feb':'02','Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11', 'Dec':'12','January':'01','February':'02','March':'03', 'April':'04', 'May':'05', 'June':'06', 'July':'07','August':'08','September':'09','October':'10','November':'11', 'December':'12'}

In [9]:
def date_comparaison(date1, date2):
    if '/' in date1:
        d=date1.split("/")
        d1=[]
        d1.append(d[2])
        d1.append(d[0])
        d1.append(d[1])
    if '/' in date2:
        d=date2.split("/")
        d2=[]
        d2.append(d[2])
        d2.append(d[0])
        d2.append(d[1])  
    if '/' not in date1: 
        d1=date1.split("-")
    if  '/' not in date2:
        d2=date2.split("-")
    if ',' in date2:
        dd=date2.replace(',','')
        d=dd.split(" ")
        d2=[]
        d2.append(d[2])
        d2.append(mois[d[0]])
        d2.append(d[1])  


    len_min=min(len(d1),len(d2))
    for i in range(len_min):
        if len(d1[i])==len(d2[i]):
            if d1[i]!=d2[i]:
                
                    return False
        else:
            len_m=min(len(d1[i]),len(d2[i]))
            
            if d1[i][-len_m:]!=d2[i][-len_m:]:
                return False
    return True

In [10]:
def compare_Ini(object1, object2):#compare les mots avec leur mot ou leur initiales
    o1=object1.split(" ")
    o2=object2.replace('.',' ')
    o2=o2.split(" ")
    while '' in o2:
        del o2[o2.index('')]
    
    len_min = min(len(o1),len(o2))
    for i in range(len_min):
        if (o2[i][0]!= o1[i][0]) and o2[i]!=o1[i]:
            return False
    return True

In [11]:
def compareInitiales(object1, object2): #compare les mots avec leur mot ou leur initiales, peu importe l'odre
    o1=object1.replace('.',' ')
    o1=o1.replace(',',' ')
    o1=o1.split(" ")
    
    o2=object2.replace('.',' ')
    o2=o2.replace(',',' ')
    o2=o2.split(" ")
    while '' in o2:
        del o2[o2.index('')]
    while '' in o1:
        del o1[o1.index('')]

    len_min = min(len(o1),len(o2))
    for i in range(len_min):
        if (o2[i]not in o1) :
            b=False
            for j in range(len(o1)):
                if o2[i] == o1[j][0]:
                    b=True
            if b==False:
                return False
    return True
    

In [12]:
def is_synVal(object1, object2, graph1, graph2,fp1,fp2):
    object1=object1.toPython()
    object2=object2.toPython() 

    if object1 == object2 : 

        return True  

    if type(object1) is not str or type(object2) is not str:

        return True

    if type(object1) is str and type(object2) is  str:
        if is_date(object1) and is_date(object2):
            return date_comparaison(object1, object2)
        if object1.startswith('http') and object2.startswith('http'):
            
            return invalidate(object1, object2, graph1, graph2,fp1, fp2,3)
        if object1=='Male' and object2=='M':
            return True
        if object1=='M' and object2=='Male':
            return True
        if object1=='Female' and object2=='F':
            return True
        if object1=='F' and object2=='Female':
            return True

        d=textdistance.jaro_winkler(object1,object2)
        if d>=0.70:
            return True
        if d<0.70:
            if compareInitiales(object1,object2):
                return True
                    
        if '<' in object1 or '>' in object1:
            return True
                   
    #print(object1)
    #print(object2)
    return False

In [13]:
def invalidate(subject1, subject2, graph1, graph2,fp1,fp2, n):

    predicates1 = set([p for p, o in list(graph1.predicate_objects(rdf.URIRef(subject1)))])
    predicates2 = set([p for p, o in list(graph2.predicate_objects(rdf.URIRef(subject2)))])
    predicates= predicates1.intersection(predicates2)
   
    for predicate in predicates:
        
        if predicate in fp1 and predicate in fp2:

            objects1 = list(graph1.objects(rdf.URIRef(subject1), predicate))
            objects2 = list(graph2.objects(rdf.URIRef(subject2), predicate))
            object1=objects1[0]
            object2=objects2[0]
            object1=object1.toPython()
            object2=object2.toPython() 
            
            if object1 == object2 : 
                return True  
      
            if type(object1) is not str or type(object2) is not str:
                
                return True

            if type(object1) is str and type(object2) is  str:
                
                if is_date(object1) and is_date(object2):
                    if date_comparaison(object1, object2):
                        return True

                if object1=='Male' and object2=='M':
                    return True
                if object1=='M' and object2=='Male':
                    return True
                if object1=='Female' and object2=='F':
                    return True
                if object1=='F' and object2=='Female':
                    return True
                if n>0 :
                    if object1.startswith('http') and object2.startswith('http'):
                        
                        return invalidate(object1, object2, graph1, graph2,fp1,fp2, n-1)

                d=textdistance.jaro_winkler(object1,object2)
                if d>=0.70:
                    return True
                if d<0.70:
                    if compareInitiales(object1,object2):
                        return True
                if '<' in object1 or '>' in object1:
                    return True
    
    return False

In [14]:
def invalidate_sameAs_links(sameAs_list, graph1, graph2):
    fp1= get_functional_properties(graph1, 1)
    fp2= get_functional_properties(graph2,1)
    
    erroneous_sameAs_links =set()
    
    for sameAs in sameAs_list :    
        subject1 = sameAs[0]
        subject2 = sameAs[1]
        
        predicates1 = set([p for p, o in list(graph1.predicate_objects(rdf.URIRef(subject1)))])
        predicates2 = set([p for p, o in list(graph2.predicate_objects(rdf.URIRef(subject2)))])
        predicates= predicates1.intersection(predicates2)
        
        for predicate in predicates:
            if predicate in fp1 and predicate in fp2:
                objects1 = list(graph1.objects(rdf.URIRef(subject1), predicate))
                objects2 = list(graph2.objects(rdf.URIRef(subject2), predicate))
                object1=objects1[0]
                object2=objects2[0]
                if is_synVal(object1, object2, graph1, graph2,fp1, fp2)==False:
                    erroneous_sameAs_links.add(sameAs)
                    break
    return erroneous_sameAs_links               
        
        

In [15]:
def extract_sameAs(refalign_path):

    same_as_list = []
    refalign = minidom.parse(refalign_path)
    
    source = refalign.getElementsByTagName('entity1')
    target = refalign.getElementsByTagName('entity2')

    for i in range(len(source)):
        s = source[i].attributes['rdf:resource'].value
        t = target[i].attributes['rdf:resource'].value
        same_as_list.append((s, t))

    return same_as_list

In [16]:
def create_erroneous_sameAs(graph0, graph1, refalign_path, output_path, ratio):
    
    sameAs_dict=dict()
    sameAs_list=extract_sameAs(refalign_path)
    
    for i in sameAs_list:
        if i[0] not in sameAs_dict:
            sameAs_dict[i[0]]=i[1]
    
    erroneous_sameAs=[]
    
    l0=list([s for s, o in list(graph0.subject_objects())])
    l1=list([s for s, o in list(graph1.subject_objects())])
    
    while (len(erroneous_sameAs)< len(sameAs_list)*ratio):
        s1=random.choice(l0)
        s2=random.choice(l1)
        if s1 in sameAs_dict:
            if s2 != sameAs_dict[s1]:
                if [s1,s2] not in erroneous_sameAs:
                    erroneous_sameAs.append([s1,s2])
        else:
            if [s1,s2] not in erroneous_sameAs:
                erroneous_sameAs.append([s1,s2])
            
            
    refalign = minidom.parse(refalign_path)
    alignment = refalign.getElementsByTagName('Alignment')[0]
    for i in erroneous_sameAs:
        m = refalign.createElement('map')
        alignment.appendChild(m)
        cell = refalign.createElement('Cell')
        m.appendChild(cell)
        entity1=refalign.createElement('entity1')
        entity1.setAttribute('rdf:resource', i[0])
        cell.appendChild(entity1)
        entity2=refalign.createElement('entity2')
        entity2.setAttribute('rdf:resource', i[1])
        cell.appendChild(entity2)
        relation = refalign.createElement('relation')
        eg = refalign.createTextNode('=')
        relation.appendChild(eg)
        cell.appendChild(relation)
        measure = refalign.createElement('measure')
        measure.setAttribute('rdf:datatype', 'http://www.w3.org/2001/XMLSchema#float')
        u = refalign.createTextNode('1.0')
        measure.appendChild(u)
        cell.appendChild(measure)
                
    refalign = refalign.toprettyxml()
    new = open(output_path, 'w')
    new.write(refalign)
    new.close()
    
    return erroneous_sameAs

## TESTS

In [18]:
graph0 = rdf.Graph().parse(source='IIMB_LARGE/000/onto.owl', format='xml')

In [20]:
graph1 = rdf.Graph().parse(source='IIMB_LARGE/001/onto.owl', format='xml')

In [21]:
output_path='IIMB_LARGE/001/refalign2.rdf'
refalign_path='IIMB_LARGE/001/refalign.rdf'
ratio=0.4

err=create_erroneous_sameAs(graph0, graph1, refalign_path, output_path, ratio)

In [23]:
sameAs_cor = extract_sameAs('IIMB_LARGE/001/refalign.rdf')

In [24]:
sameAs_err = extract_sameAs('IIMB_LARGE/001/refalign2.rdf')

In [25]:
set_cor=invalidate_sameAs_links(sameAs_cor, graph0, graph1)

In [26]:
set_err=invalidate_sameAs_links(sameAs_err, graph0, graph1)

In [27]:
len(set_err)

518

In [28]:
len(set_cor)

0

In [29]:
TC=len(sameAs_err)
TP=len(sameAs_cor)-len(set_cor)
TN=len(set_err)-len(set_cor)
FP=len(sameAs_err)-len(sameAs_cor)-(len(set_err)-len(set_cor))
FN=len(set_cor)

In [30]:
print('TP : ', TP)
print('TN : ', TN)
print('FP : ', FP)
print('FN : ', FN)

TP :  1416
TN :  518
FP :  49
FN :  0


In [31]:
accuracy= (TP+TN)/TC
recall=TP/(TP+TN)
precision=TP/(TP+FP)

In [32]:
print('accuracy : ', accuracy)
print('recall : ', recall)
print('precision : ', precision)

accuracy :  0.9752899646999496
recall :  0.7321613236814891
precision :  0.9665529010238908


In [33]:
TC

1983

In [34]:
sensibilite = TP/(TP+FN)
specificite = TN/(TN+FP)

In [35]:
print('sensibilité : ', sensibilite)
print('spécificité : ', specificite)

sensibilité :  1.0
spécificité :  0.9135802469135802


## Evaluation

In [40]:
graph0 = rdf.Graph().parse(source='IIMB_LARGE/000/onto.owl', format='xml')

TPl=[]
TNl=[]
FPl=[]
FNl=[]
accl=[]
recl=[]
precl=[]
sensl=[]
spel=[]

for i in range(1,21):
    if i <10:
        graphi = rdf.Graph().parse(source='IIMB_LARGE/00'+str(i)+'/onto.owl', format='xml')
        print ('File : 00'+str(i)+' :')
        output_path='IIMB_LARGE/00'+str(i)+'/refalign2.rdf'
        refalign_path='IIMB_LARGE/00'+str(i)+'/refalign.rdf'
    else:
        graphi = rdf.Graph().parse(source='IIMB_LARGE/0'+str(i)+'/onto.owl', format='xml')
        print('File : 0'+str(i)+' :')
        output_path='IIMB_LARGE/0'+str(i)+'/refalign2.rdf'
        refalign_path='IIMB_LARGE/0'+str(i)+'/refalign.rdf'
        
    ratio=0.4

    err=create_erroneous_sameAs(graph0, graphi, refalign_path, output_path, ratio)
    sameAs_err = extract_sameAs(output_path)
    sameAs_cor = extract_sameAs(refalign_path)
    set_cor=invalidate_sameAs_links(sameAs_cor, graph0, graphi)
    set_err=invalidate_sameAs_links(sameAs_err, graph0, graphi)
    TC=len(sameAs_err)
    TP=len(sameAs_cor)-len(set_cor)
    TN=len(set_err)-len(set_cor)
    FP=len(sameAs_err)-len(sameAs_cor)-(len(set_err)-len(set_cor))
    FN=len(set_cor)
    
    print('TP : ', TP)
    print('TN : ', TN)
    print('FP : ', FP)
    print('FN : ', FN)
    
    accuracy= (TP+TN)/TC
    recall=TP/(TP+TN)
    precision=TP/(TP+FP)
    
    print('accuracy : ', accuracy)
    print('recall : ', recall)
    print('precision : ', precision)
    
    sensibilite = TP/(TP+FN)
    specificite = TN/(TN+FP)
    
    print('sensibilité : ', sensibilite)
    print('spécificité : ', specificite)
    
    TPl.append(TP)
    TNl.append(TN)
    FPl.append(FP)
    FNl.append(FN)
    accl.append(accuracy)
    recl.append(recall)
    precl.append(precision)
    sensl.append(sensibilite)
    spel.append(specificite)
    
    print (' ')
    print(' ')

File : 001 :
TP :  1416
TN :  526
FP :  41
FN :  0
accuracy :  0.9793242561775088
recall :  0.729145211122554
precision :  0.9718599862731641
sensibilité :  1.0
spécificité :  0.927689594356261
 
 
File : 002 :
TP :  1416
TN :  522
FP :  45
FN :  0
accuracy :  0.9773071104387292
recall :  0.7306501547987616
precision :  0.9691991786447639
sensibilité :  1.0
spécificité :  0.9206349206349206
 
 
File : 003 :
TP :  1416
TN :  502
FP :  65
FN :  0
accuracy :  0.9672213817448311
recall :  0.7382690302398331
precision :  0.9561107359891965
sensibilité :  1.0
spécificité :  0.8853615520282186
 
 
File : 004 :
TP :  1416
TN :  505
FP :  62
FN :  0
accuracy :  0.9687342410489158
recall :  0.737116085372202
precision :  0.9580514208389715
sensibilité :  1.0
spécificité :  0.890652557319224
 
 
File : 005 :
TP :  1415
TN :  505
FP :  62
FN :  1
accuracy :  0.9682299546142209
recall :  0.7369791666666666
precision :  0.9580230196343941
sensibilité :  0.9992937853107344
spécificité :  0.8906525573

In [41]:
index=[]
for i in range(1,21):
    index.append(i)

In [42]:
import pandas as pd
import numpy as np

In [43]:
#ar = np.array(TPl, TNl, FPl, FNl, accl, recl, precl, sensl, spel)
df = pd.DataFrame(TPl, index = index, columns = ['TP'])

In [44]:
df['TN']=TNl
df['FP']=FPl
df['FN']=FNl
df['accuracy']=accl
df['recall']=recl
df['precision']=precl
df['sensibilité']=sensl
df['spécificité']=spel


In [45]:
df.head(20)

,TP,TN,FP,FN,accuracy,recall,precision,sensibilité,spécificité
1,1416,526,41,0,0.979324,0.729145,0.971860,1.000000,0.927690
2,1416,522,45,0,0.977307,0.730650,0.969199,1.000000,0.920635
3,1416,502,65,0,0.967221,0.738269,0.956111,1.000000,0.885362
4,1416,505,62,0,0.968734,0.737116,0.958051,1.000000,0.890653
5,1415,505,62,1,0.968230,0.736979,0.958023,0.999294,0.890653
6,1416,519,48,0,0.975794,0.731783,0.967213,1.000000,0.915344
7,1416,506,61,0,0.969239,0.736733,0.958700,1.000000,0.892416
8,1416,492,75,0,0.962179,0.742138,0.949698,1.000000,0.867725
9,1416,515,52,0,0.973777,0.733299,0.964578,1.000000,0.908289
10,1416,496,71,0,0.964196,0.740586,0.952253,1.000000,0.874780


In [394]:
df.to_excel("data.xlsx")